In [104]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader ,Dataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [105]:
torch.manual_seed(42)

In [106]:
# check for gpu

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f' using device:{device}')

 using device:cuda


In [13]:
df=pd.read_csv("fmnist_small.csv")

df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [16]:
df.shape

(6000, 785)

In [116]:
x=df.iloc[:,1:].values
y=df.iloc[:,0].values
x.shape

(6000, 784)

In [24]:
x_train,y_train,x_test,y_test=train_test_split(x,y, test_size=0.2 , random_state=42)

In [117]:
x_train=x_train/255.0
x_train=x_test/255.0

In [118]:
# create a customdataset

class customdataset(Dataset):

    def __init__(self, features ,labels):
        super().__init__()
        self.features=torch.tensor(features , dtype=torch.float32)
        self.labels=torch.tensor(labels , dtype=torch.long)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index] ,self.labels[index]

In [119]:
train_dataset=customdataset(x_train,y_train)
len(train_dataset)

4800

In [120]:
test_dataset=customdataset(x_test ,y_test )


In [121]:
train_loader=DataLoader(train_dataset ,batch_size=32 ,shuffle=True)
test_loader=DataLoader(test_dataset ,batch_size=32 , shuffle=False)

In [122]:
len(train_loader)

150

In [123]:
# create nn class

class mysimplenn(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model=nn.Sequential(
            nn.Linear(num_features ,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64 , 10)

        )
    def forward(self ,x):
        return self.model(x)
    


In [124]:
# set learing rate and epochs

num_epoches=100
learning_rate=0.1

In [132]:
model=mysimplenn(x_train.shape[0])
model=model.to(device)



In [127]:
# LOSS function

criterion=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [134]:
# training loop

for epoch in range(num_epoches):
    total_epoch_loss = 0.0

    for batch_features, batch_label in train_loader:
        batch_features, batch_label=batch_features.to(device) ,batch_label.to(device)
        # forward pass
        output = model(batch_features)

        # loss calculation
        loss = criterion(output, batch_label)

        # clear old gradients
        optimizer.zero_grad()

        # backward propagation
        loss.backward()

        # update weights
        optimizer.step()

        total_epoch_loss += loss.item()

    avg_loss = total_epoch_loss / len(train_loader)
    print(f'Epoch: {epoch+1}, Loss: {avg_loss:.4f}')


IndexError: index 3793 is out of bounds for dimension 0 with size 1200

In [ ]:
total=0